In [3]:
from math import log as log
from numpy import zeros as zeros
from scipy.special import gammaincc as gammaincc

class ApproximateEntropy:

    @staticmethod
    def approximate_entropy_test(binary_data:str, verbose=False, pattern_length=10):
        """
        from the NIST documentation http://csrc.nist.gov/publications/nistpubs/800-22-rev1a/SP800-22rev1a.pdf

        As with the Serial test of Section 2.11, the focus of this test is the frequency of all possible
        overlapping m-bit patterns across the entire sequence. The purpose of the test is to compare
        the frequency of overlapping blocks of two consecutive/adjacent lengths (m and m+1) against the
        expected result for a random sequence.

        :param      binary_data:        a binary string
        :param      verbose             True to display the debug message, False to turn off debug message
        :param      pattern_length:     the length of the pattern (m)
        :return:    ((p_value1, bool), (p_value2, bool)) A tuple which contain the p_value and result of serial_test(True or False)
        """
        length_of_binary_data = len(binary_data)

        # Augment the n-bit sequence to create n overlapping m-bit sequences by appending m-1 bits
        # from the beginning of the sequence to the end of the sequence.
        # NOTE: documentation says m-1 bits but that doesnt make sense, or work.
        binary_data += binary_data[:pattern_length + 1:]

        # Get max length one patterns for m, m-1, m-2
        max_pattern = ''
        for i in range(pattern_length + 2):
            max_pattern += '1'

        # Keep track of each pattern's frequency (how often it appears)
        vobs_01 = zeros(int(max_pattern[0:pattern_length:], 2) + 1)
        vobs_02 = zeros(int(max_pattern[0:pattern_length + 1:], 2) + 1)

        for i in range(length_of_binary_data):
            # Work out what pattern is observed
            vobs_01[int(binary_data[i:i + pattern_length:], 2)] += 1
            vobs_02[int(binary_data[i:i + pattern_length + 1:], 2)] += 1

        # Calculate the test statistics and p values
        vobs = [vobs_01, vobs_02]

        sums = zeros(2)
        for i in range(2):
            for j in range(len(vobs[i])):
                if vobs[i][j] > 0:
                    sums[i] += vobs[i][j] * log(vobs[i][j] / length_of_binary_data)
        sums /= length_of_binary_data
        ape = sums[0] - sums[1]

        xObs = 2.0 * length_of_binary_data * (log(2) - ape)

        p_value = gammaincc(pow(2, pattern_length - 1), xObs / 2.0)

        if verbose:
            print('Approximate Entropy Test DEBUG BEGIN:')
            print("\tLength of input:\t\t\t", length_of_binary_data)
            print('\tLength of each block:\t\t', pattern_length)
            print('\tApEn(m):\t\t\t\t\t', ape)
            print('\txObs:\t\t\t\t\t\t', xObs)
            print('\tP-Value:\t\t\t\t\t', p_value)
            print('DEBUG END.')

        return (p_value, (p_value >= 0.01))

# Your binary data (as a string)
binary_data_alice = "10111010101101010000001101001011101101000100000011111011010001011101111011101001101010000000100101001010011000000000110100100010111011000101000100000101001011011101110101010001111011001011011010000101101100111010011110111000010010001001000110110001010000100100000100000010110001111101111101110100010101110100111011001100011101100100111010011110110111000011010101001100000100110100001100101000101101100010000011010110111011010101011100110011111010000101111101011011011111110010000011110111011001011101100011000111000111000111110001000101100000010100111000100110001111000101101111010001110110110011010000101011100101001010010010100011001111101010100110000001100010000011010100000010010000000110000111101101010010101001001010101100001100011000011011110000100000000111110010110110000111111100101011011011010000010001111110110000101001010101110100111010101010000101101101000100100100100001111111011000010100101011000001101010010110010000000110110110001110000111010011011100110111000101110001110001101111101001111100100111100100001110001001011101010000001110100000011010000111010010000011000000011101001101111110011000000100001101111101101110011110011011011010011101111011100010111111000111001110100011011000100010011100100011101101001001010000001101001101111010111110010100111000100011011010001110010100101110110011110110100100101110001110001101110110101101100000000001010110111010010011111011001110011000111011010010101100010101011011001000100100010110101011010100001111110010100011000100010111000101110111111101001100111110101010000001100101011111010001111010010110001011100100010110010100100100111101111001100111111011001001010110000110010001000110011101000001000000101011110110111110001010001000100100111110101111101010100100000001110101100010111100000011000010100001010000110101011111100100001010110001110000111010111100000110011111100001001001000000110110010010010110011000100101110000000111000010000010011010000001100100010111110110000000100110111010111000011010000100100111001111011101110000010001010110011111001001010000111101000001000100101101010011100010011001010000010000101110010110100010110011010101010010101000111011010000100100010001101010111000110100100101001010010011111100101000100001000011100011010000101100011110111100101010111000011110011101010001111001101000011010000011101111011000010111010000010101010111101011001010110010101000001101101000100001011111110011110101110010011010111110011101010101100010000011000111000100101111100001101010000000101110010010011100100100110110011110000111110010011101111001101100001111110111100011111110001100010110111001100010101010010010101010011100100100001001010110000110111011000101001001110001001001101100000010010001000010111001100010000100000101001111100010110101101101110111111001011100011110010111000001000111100110000101010011111011001010110110111011001111111111011100011100000100100000010100110100101000000101000000111111110101110011010100101111001110100011010011010011110110111001000010101100001110000111001110111011111100000100110001000010000010110100110111000000010001111000001111000000010101000001011001101010010011010000110010001110010000010011010011110101110001010100010010000000110010000100111101110010100111000100100010010100100001001110000011111001011101110101000001000110100100001100001111110010000011101111011110000001011010100110010000010100001110011110001101010100100111000010011111001101000010101101010110101011100110110001100111000101011100110111001000100010011000111110111110110101110000100100000111100010101111110011000001100001100100101111001111010100110010101000111101001111111110011100010010110100100101110011011111110011111001000011010011101100010001110110011110001010011110001011100100010001010010001001010100000100000011101010001100110100110010100000011101101101010111011010100100010011011110000100110100100001011000010111101111001001000100010110000011110010001101001100101000111011111000100110100110011101100000001100110110111101000100000101000111100001101101010001010111101000100101111101111001000100110001000000000000111111100000001001111001011001100010000111011011010010100000110010111001001001100101111010101011101110010111111111100010000111000001010011111111101000010010001011101001001010110111101110100100110111010100011010001101100111101100000100011111010101111010111111100010100111101001000001111100111101110010101100110011010001100010101101101100001000110011110100110101110101001011000101110000101011110000100101101011100101001000000011000010001100000100100010111111011101000000100000101100101110001000111010011000000111001101011100110100011001100011101000000101000100110011000110001001010111100001011110110101000011101100100000100000111001010100011011010111000100001010111101000111010101010111100110111101111011001101001000001011101110011111010111110010001010110110010101100101011100100110010011011010110000101110001000111011111001101000111110001110110101110110101001011010010000100110001110000100010001110111110011111110001000000000100011110101001110001111111110110111111001100101100111011100100011101011100101000100000110010100101011000101010100011101111001010000010001001100011001111011011101010101010101100001000000110101011010101111001000011010011010110110000101010101100111010000111011001001000100111110100100110101011001000001111000000011000110101000010010110101101010101101010001011101101000000110000010001011001110110110100001101011100000110010011111010110100110100010111100101011110100100001000110100000000001111111111001101111011101111011010101011101100100011111110011011100010101000001100100110000001100111011010101111101100010110001001010001001111010111111110011111011110101010010111110101110011011000000100000110011000110101001100100001001001010000011111011000111000010000001110111101111000101100001000010011000000111001000111000111000011111000111000010101001111101011000001110110101111000000010111000000110111010111010111111111010100010111110101110011000110001011111000111011001011011100010100010011011000100000010101010000101000001011010001100010010010101110001111000001111011100111100001000000101111010000001100001000001110001100001011110011011000011101010111110100011010101011011001001100001100000111000101111011010110111110011101111010011100110010100101010110001011101100000011110110001010000011100010101001100100011110000100101011010110010011101101111110000000110000001000001100110011110101101011110111011101100110100100001011100000000100000110111011101011001100011111010100110101100010010010000000000110000110111010010010111010110010110110110100010011111010001111001110011110111101010111100010101110111100010010111001111010000011111001000111011010010000111001110111000011100011011010110100101001010000110000110001000001111011000111110001110001011110000101001111111110100101000101000011000011100111000010101111100000101111000100000110110010000110001110010101001111001111111000110010011101100000000000111100001101101100100000001101000101111010111100000111001101111011111110111111011010010000000111011010111100111011101010111011111010111111011101110000010011111000111001011010010101110010101100010101000101011000111000101010011111101011101110100001100001010101100100000001001101010110001100000010011001101010111100010001101011110110101011011001110000100001111100110100100100000100111111100000011010111101100101101000111011110011010111001001110001101110001100001101011011001010110001011000011110111010011011011100110111011001111011101100101111111110000100001110101100011111000101011101001100000000010101001100111111101011001000000000100101100010001101100011001111001101110111011110101001010001010111101110110101110001101100000100110111100011001010101000010000011101111000010111100111110110101010111000110110100111110110001011100101100010000100100010000110000111110000010000010111010001011010011100011100010000000011100001011110100110001011111010100001110100110101101111010011111001110100011110110100001011000110101110010100111000100111110110110011100111100100101010111000000000011010110011101111010000101011001100110101001101111010101110011011011000101000000001001001001000110110101001000110101001000010110101111101001000101101110110010100000000011110110000110000101111001000010011101010110110110000110011010010100011110111101011101011111110101010110000001001010001001010000010111010011000000000110000100110110010011001010100000010011000010101110011000111100001100110011011100100001100000000110100001000001001100111010110011110101100100001011010111010110001010001100111100011000101101001000000010000001011101001000011101110110100100100100111001000111111110101011111111110010110001000101011011010011011001001111010101100111110101101010101000010010101110000000001110101001001101110001110100001101010100011010110000000101010100111111110110011011111111010010011110001101111110010111101011010110110001110101110110101100001100100000010011010111000110110110100100001011100000010100111000110110110101101100010010100111000111001001001001111110111100100000010010011011000111110001000001000110000010110101100011010101101110100100101000101000000001111011011100000011110101010111001111000100111111101000001000001010000111010110111010001010110010100011101011000100001110011101000000100010101100000010000111000100010101001110110111111111011011110000111111010000011110100010000110001111111110110110101011100101110000001011000001100110010110000101001010100010110001010101001111101100110001011110001000001111100010110001110010101101101111110100010000111010010010001111010110100000000100001110101100101000101000000101111001101001111100111101100001001100101010101000111111111111010101011010110110011101111010100100010010011011110010100101010111001001110011011010111011011111110100000101100000101000011000110000010011111100000111010110101000010100011110101110110011011010001101001000001000111101001100110101111011011000101000100111000111000111111111110100100001011010111110011101000100011010101101001110000100011110100111100001100000100100101011010011101111010110101101010111101111111110111001101100010000101100010011110001110110111001001100110111000000000100011001100011000111101000110100000001011001110010010000101011101011111011010010101100001001010010010101100010010001110111100101110111001010001101100111011100110010101011010110011011100010111110011000000001011110001010010001111111101010000100000101100100100011010000010010101101101100000101101010010111001110101111011000110001110010001111111100000010101001100010000011000010010100010011101110110100001010111010001111100010001000011001101101001010110110100001110001000101110111100000011110110010100001110010100000001000000111110011110001001000100110001010000011011000011000110110010001000011101100000001010010000000101010101000000110100010001001000010111101011001110001000010111000011000011001100001110000010001011001000100101010100000110011010100101010111110100110100110001000111010000110101010110011110010111000111101001011101100111000001010001100111100010001101001110010001111100111110010101110010100110111110000010001011000110001010011001000110000001101001110001011101001001110000001010101110001010101010101011100000011111101010111111010010001111011010001000000010000001111100110100001010000011010100001110110010010000110000100000111010100101100110010100010110110010001001110100001101001010111110110001011110010000001000001100001011000001010001110001101111010011001000010101011111111001110100010011011100111100101101101011110110111000111001011000010111101111010000011100101000010111011000010011001001100111000110100100110000100111111000011101010111110110000100111011110101000111010011011000010001110111111001100100100001100101001110010000110111100000000010000011011100000100110001100001001101010100001001110110110111000001000010001101001000101010011011100101101101111011011100011111100111100110110100111101101000111001011111110001111100011100001111111010110010011100010100100110010110101111000010000000010010100110110110011011111011110101100110000111100000111010111001111111000111110010100000010101000100101101011110100000000111111011111100001110010100000010001111010011111111001000100101101100000000101111010011101010101111111000011000111100011011010110001100010100010110011111001011000011111000001110101101111111010001011101000111101011100111110100011110100110011010100000100010001110100001111001001100100100111000011110101010110100011000100000110000000010100010000101100001010000111011100000001011100101001000100000110011110100011101111110111010110111111100001000101001010111010110010110011100111100111000001001110001001011011110101110000000110110000111110010011101101100000110001010100100000101010001010100100000101110100100000100001101000011010001101011011101100011010011101010100011011001110010100010011001010101111100110101100110100010010110100011000011110001101010011010110110001001000000111001101101001110010101110111000111001100100101001001101000111000100101100100101001011001010101111010100000001011010001001100000111111000000110100111100000001010101011000111010001110000011111010111100011001110000110100111011010001101010111100001111001001111011100100001110101010111010000000101100100001011101011100010001110011001100001011010100100110000110101000011111110101010111101101100000110010001010010110001000100100110110010111110111110111111111101010110010001001101100100100010001000000110011001010011001000101000111100101101010110010100110001000100000100000111100011101100010111101100011101010011101011010100110010101000110011001101001000011110001000010110011111001001101010001010011110001000110010110100100101101001100000001010111101111000001010101110111001001000010101010101011101011110010001111011111101111000010001011011101110100011010111011111010011010011001100100111011001100001110011001010011010101111010010000111110000100110111111010111010010100001000101010110110100100000010101101110110100101001100111010001111110100101001100011100010010111101101111010001100100100110011111010111001000000110111011111110010110000110010000001101101100010110001011011101100000111010001000011101010110110010000100100101001101000001101100001011001001110101110100111011001100011000010001111011011010011000001001011111100101101011111011000100111101011001010001111011101011010001101010110010101010001100110010000101101101010010000011110000011000111010010011110101001110100000000011101011101010010011111010000001000100101011111110001011111001111101000110111000100111111111011100001100110101001011110011110000000010101111001110011101011011011001010100011001011101101111101100011100110110001000101101110000011111101001011000001001100011001010001010100001101100100110010110011011010100001010111111000111111000011100010011101011011111111001100010000001111101100010000110101011101100110010111100101110000011101110010000000000001000100101011010100010111111001010010110011000001010001110111100011001010110010110011100011100100110010110011010110000111101110001111001110010111110011001101100110101101110110101001011101010101101100000000010101000000010111110100000110001010111111100101111011100110000110001001000100111001100011010101011000001010011010101111111000110011110010000000010000010101010000111101101110001010011110110010000001010001001101110010011011011101000010110011111111010001001100110001100101010101101000110001110010111001010000101110100111000000011101011101011011011010101000011011111000101000101010101011010110011111101010110101010110010110011110001111111011001100010011111111101101010110101101000010101100011110100111000100100001010001011100001101000000000010111110001101011000001011111110011110010100010101100111111101000100000011001010100100110100001011011101001000010111100000101011011010111101101010101100100111011000001000011011010010111011111011101010001011001101000001001100001111011001110101000011100110110111110101101000001101101111001010000000111010001101111100101011100011100010100110000101011010011000111100110010011001101011111011011001001001111111110011101101101111110000010111000101101110100011110111010110001100100100100010010100101110010101111100111010110010101110010000011010001000001100111100111011110001111111001011000000110011110100000100101100101100011110111010101001111101111101110101000101011111011101111101100100010001110110010110101110110100000010000111110100010010010110111110101010011101000010100001100101001101010000011001111100010111100111011101111100100110000011110010011000010100111111101101100001010011110100000101000111110110111100100011001111000100100010100011000001011011010010001110001101110110000100001101111001100111010000010001001100000111011110100010011001001101111000011011000001011000010101011010000011001011111110101101001100101000000010001101100101011000111011100101100001001011011110001101111111001010000100010110100100111010101001010101100100111111000000101010011111101100011100101100001111010111111011010111110111110100010111010110110011011001110011101110000110001010000110000101001100011101110000000000100111101010000010001101110111111111001110000101010011001010000001111000100001100100101001111110111000011100101001000110001000110011111110001101101100011111110100000010000100110110010000010010010000011111011111100011000001110100101011110100000010001000100111101010010000101101100010001011100011100110000111011110110101001000011001110010100000110101011100001010011011011111110110011000100101101011100101001010001100010110010101111000000000100001101111001101111110000101001010010100111101110100110000110101110010001011010000011010000100110010001011011110111111110111110000011100001000000100010111101100110000010011010110001010001011010011101100001101011011100000110101110000010101000010110000000011011111000110110101000011110110011010010000111010100000100000100001011110111001111110011100101000000100111000000010100011110110101000100001111001101000011100000011101000100100010100"  # Replace this with your binary data

binary_data_bob = '10100010101100001111111010010001111010001111100000100100110111100000000011011010010001011000001000011011011011111100001000100101011111010100010100001111101000111000100100011010001010101010101010011010010111100001010010000101100101101111000000001100111010010111100010100101001101110101100000001100010100011101000100100100001000000101101011010111110100110111000011100010001101100110101011000100101000000111111110000100001011101101000010010111101001011001110101000100001000111001111110001011000100110110011101100011111001011111011001001101001100111100101001000101000000101010011010011000001010001000001011010111001011001010000011011101100110101010110010000110011100010011111010101001001001011110010101001011011001101000011010001101111001101100001100100011000110100000001100000000000000100010101110010000001110101011000001101100001001100110110011110011100101001001010111010110011001000000111000011000100101010010100110110010110100100000100110010000100000111011001001110100001101000100011100000001011110110110010010010000001111011011100100010000110000111100110111000101101010111011010111111010011110001000000011010000011010011100010001001110011111100111100101000110111110110000111001110100001101010110000000110000000000001101101001001000000010101110100101011110111011100001111101010100101101010000001010011001011100110110010000100010000100100001110000000111100100101111101000011011010011110111011100001010000011010101101101011111111001100100010010000001000000110010100100110101011011011111001010101101111101111011000111110010001011011110010001100011001100101100100110010100000001011010110100001010110010100111010011010000101011111011001010010010010110111000100011100001010001000110101110001110100100111101111000111111001010010110010100011001111101001101001111001111011000111000111101011010100010101001010001110011011011110101101011100010000111111001000000100011110001011010001110010100101000011110110000101100011111101110001000000010100111010101001010010101101100110001010011010110101110001101001000001100001011110000010100110100101101010101000110111001111110110000010010111000110001001001001100101100010111101101010001001010001001000000111111111000110101010010000101110001110001010111010101000000111110100010001110100011010101001100011011111000100110011100010110100001111010001000000111000000101001110110111111010010101111000011011000010110111111001001000100010100010010001111010001101111001001111100001000011110110110101111100111010111000001001010001111100110000100011000100100001101000110111101001000010100110110011110111011001101001111000001000010011111001011000111010100111011101010100111111011001001100100101100101011001110110001010111000011000000111001001100101011101111010000010010000100000010000100001001101101100000010100010000111111011110110111110000101101000001010101010001000100000110000111001100011101010000010011010110101110000101100101011000001010001001100100110001110110010001010001011000110000011101000111011101100111010010000111101110011100011011111100011011111011101101110000111100111011110110011110100001101111111011011100000111011100100010101110000011101011000110111010000000000001000101101000111000101101100011011000101011110100001010010010101001000001110100010100111101100100101110101011111100101101010100100011000100011101011111111000001110000000110100100001000000111001110000110000011101111101111101110011101101100010010011101110110001100000101001111111111100101010000110001101100011010100001110110111001110101010110111101001101100001100010110111000110000101000011101000000001100011101110100100010011111010000000011110100100000000010111011000000100101010000110110001000011001001100110010111101100111010100111111010011101000101001000111000011101000101000110011011011111001111011011100000001001011101110110001000001101110100100100011001101110011111100000010010100100100001001000110010111011011100000010101101110010110011111001010011111010100110010100101001010010000111010101110110111000000101100101110010101010001101111011100010100001101010010000011000011011110101000001000010110001001011001011000101100000010101001000100100011110000111011010000010010001001011000100000001011001001001111000110111010101011100010111010000001111000000111110011000110011001101111001001011101011011010111101101101010110001110001011001000001001101110101000010010100011111010110010101010000011000010100010001100111101111101010100001110000010010010010110101110010010010000100111111011100011000010101110101011001100001100100101000010100001001110100111001001000110000000010000001011010000111100000110110010101101010010110110100001101011110111001100001100001101010110111000100001011100001100101001111101101010101111011100110111001000110101011010011001100101010100010111011010011101001100101000111001011001011100001110101001001000100010111100111001100010100001110000111101010101110101010001000111010101110111110100010100010010110110111001101011010001011100110010000110100100101110101101001010101100001010100011101110101001001000011001111110000110100010011000101101110100100010110100000010001101101010000010010001000111000000110101100000111011110110000110001010100110100100100001100010111101101101100100001101101001100001110011100100001000000100101100011110000000010010000101111101010111111100110000000110001100011011001111100101001000010110110111110000011001011110001010011001111110101011011011111110111111011000111101100001111110111001001100001110001000100000110011011111001000111111010001100111001010100011000010000110001011101111111110001010010010000101011000010110011101110111010111101000111001001011111101011101110100111111101011010011010010110001000110100001000110011110011111011001001110110011000101010100010101100111001001011100100000100000010110101100011101100011000000101111010100101000010110101111011001101111000100011100010011001101110101101001001011110101110110011010010101001100010000110001011011001100001001001100000001100100000110000010011100011010001011111000101101111110101101001000010110111110001101010011000011100111000010101110001110110110001101001100100111010011100101110111110011111000011011100010001111100100100100011000000100101001111101100011001101011011100010101111000110111011010110111100111010000111000001000001111110001010110000000100010000000101101110111000110010100101101101010010000101100011111101001000111110111011100111011101010100110010010011101010100010010000100110110001000011010100110110010101100010101001000001110000101111110110001011110011111100100000110011001111001001111011101110000001000110010101001010011101001100010011111011010001000110110011111110001100011011011101010000100101110010101100000111001011001000100101001010100111100101011001101111101100110000000101101100100100010000010100111101010011011010011000001010111001011000011010001000000111111110101001101010010000011101100001010110110101011010001100011011111000111010011011100110000000100111100001000000111001000110101110010010111110100101010110100110011010011100010010011011010110100111110001000010101001010010010101001100010001100000011000001111101111101011011010100010001100111110101101000100011110000111001101001010101111110001100110111011010000101001010110110100100011000111110001010011010001100111011000010100011110100100100100100010101000110011000000000000100011001100010010111001111101010111000110011100001111001011110010011101100011101110000111111000101111001000010100011011010010001011110101010001100111110100000000000110000010001100000011111000011100110111001000010001011100011100000001011011111010101000100100011001111110011100011010000110101001101010100001101010011100001000111101000100101011011011000001010010100101101011100000010100101110000110001000110010100000111101111001011010100101001101010001101000100010011000011110010101001011001010111101110000101000001110110000010111000111111100000001100100000100010110000011100000101001000101011001110101101111010001100111110001111101000100111010101110111111110010100011011111010000111010110110100100000001011000011101001100100011011111001001101011110010111110001010101100001100000111111110000010000001011111010111101100000100011010110000100010000110000101100011100111001100111010100101101010110100110000110111101011110111001110000101011001001100110000100011010110001011010010000000001010011011100011110110000100000000010011110001111010010000001001000000000110011011001110100111001101110101100011111011111110101001001010110111110110001110010100000010011101000110100001000110010001110001100011000110101100010101011110110010000111111100011111011111011001001101101010110111001001110010001011101011101110000010001010011100101101100011111001000000100101001010101001100011111001111001111010000100110110100001000110110100100100000000111110110001101001011000010111100101111101011101000000010111011011110110000110111011110001010100110101011111100110100010001011100010101100100001101101011000100000011011000111111110101111010111010001010111000110000101110111000000000001110101010001110111111001100110110111111100001100110111101111100101100000101100011010101100111100111100000110100000110000000001100111101001001000011000100000001110100101011000000111101000110000010111111000011101101110000101111010100010010001001111011110110010111011000100101000000111000111111110101100101100100100010101000100001100101010100011011001001110011111001110100110010100011100111100111001010001000100011000001000011001111101110100000010110111101001100101011111100101101011101111100000010011100110010101110001001010110101001111010000010110100101100011100000000010001101000001010110111011011110110110011110001000010011101001000011011010011110110110111100000001110100001110111101111100111011101010100010111100000000100101010010000110110010001100001011111001111111010010001000101010100100101100000000000110101011100101100110100100001101000101111011111100101001100001110000001101110111010100001011110000000101101110100110010100110010000111110010010111110001101011111111000101001101110011111000000100001000110000110000101110100001110010101101111111000000000100010100100000010000100001100001101001000010000000001000010111101001011000110001101011001001111001010111011100010100000001010110111101110100100100010000101100111010011011010010011101011000111101000111001100110111010110110000001101001101101111111100111111110001001000011101001001110011001011000010001110001101110100111010001001100010001010111111011100111010101101101001000111001011110000110111010000001110101010101100001010100110000110000100010101111001110001010011111000101101101011101110000100100001000011010101111000101110110000010110001111000101111100110010111110100100001111100010111101010101110001000010101011101000101011011010110010000101011000100000110011000010001101101001000110110110110011100111000101011111001100111000100011001110111011010000011110101100011110101000110111001101101111110011101001110010011011110000101001101111110010011100000110011011110100110101011111011000001011000110010000110111101100000110111010000101101011010000011110101010011000000010000101001110010100011011010110001011101111100100000101001010101100001100011110010000100001111110100110001110001101000001001010110111011001100110010000101010101001111111000010100001001100100010001001101000010100110110100001110111100001110111111111101100011110110010010001101001111100101100101010001000101011010100011111110110011101110001100100000100111111111101000111111101010000011010000010010000001110001011011111110100111101010000110001000111010100011111010101100100000010011110011010000010110011010001001001100111111100100001001110000101001001101111011001000101011000101000110111001000100010010011010010101010111000010100001010010000001011010000111000100101001101000010011001000100110111100000010001110010000000111101000000010000011111010001100111010011001000000001110111011011110000111110101100010011101010010000000101000001011011001000100010101111010000001001001111010110011100001111111110011100100011110101111010110000110111110001000011010101001011001010000011110110111110010001010110010010110010001001010010110011100010100000110010110000110001110101100101010011010011100010100010000011101011010001000010110110010011001110000001111101111101110000111010110010101010011001011110000000100101010110011011001110111010001000011100001010010011011101000110100110010011111010010111000110110010110010001011011101001000000110110101010110000000010010010110101100001100010011111100000000100001110100000011010011000111110101011000101001010000000001000001001101100100000011111101100000001110110001011101111010101100101110111011101100000000110010011010110010010111100011111000010010001011010100111001100100001000100111010110111101000010010000111010101101111011111000101100001101000010010111011110001001100001010110001001111001101101001101111101010000001110010111110100000011000100111111100100000000000110101110110000011011100011010101100110110110011110110010011111101110011000000010010001001001011010110101111100001111011100010111100100101000001100010010011110110011011011110000001000001100001010000000101001111100011011000000110011000110011110100101001101010110000010011001000010100111001100000000111001111111111101010110011011010010111110111100010011010111001010000110001101010101011001011100000010100101100010101001000110100010110000010010001010001011000000001010001111001101110111110110001111000001101000001010010000110000100110011000001110111111111010010111110111110101100111110010110000001100001011010111010101001111011100000111011011000100101001010111110011101011000011101101111111100011000010000000011010010010010001000100000011110100100101111100010111001111100111000111011101001010110001100001100000010111110010010110011100101110111011001010010101000000110010110000001110001100000000110011100001000100011100011001000000010001010101010010001110011101111001010101011010111110001001000000111001011001011111101010110100100111001001100001001011100110010011100101000000110101111101001100000011010000001001010100110100100111100011000000110101111100101100011101100100110100101101010001010000100000010011010001010110111110011010111010100101111111111000110000010011001110101000000110100000001110001100000011100010010000101111101110110101110000000101111101000001001000100110000100111100100101000101000011010010010111100000000110110011010011000111001101111010000000000001011100011000011001110001001001000010111001101010001010011111100111011100000111000011000100010111010011000101101101010010111101111010001011111001100110101110101010101000100000000110111111111001110110011101110110111011000110111011100101100000011011100001100111000110110000110010111110001100100001100000111111110011010111001101001000001011011101000011111001010000101000001010010010010010000101011110010001011001100010101101110110000001000101101010111000101000111110111101000000101110101000010010011100111111111010110111000010011000100111011110100011000111001011011110010100000001110100000100100110010111110100101010000010001101000000011001110110100011011001101011100000011111011101001011100010011111100100100010001001010101001100001100011110110110100110001110000010111010010101010001100100111111010100111001010001111110011010110001111010110100101001100100000010100100110111000101101001001000000110111100000110100001011100000000111101000101110001010001000010110100010100000111000000110110000010011001001010000011111100011100101100010100110010111001010110111100011001010101001000010111000010001100101101101100100001101010101101000100100111110010110001100100011001100110001011111011001000101011110000100011000101001010011100111101000100110000111101110001000110010111110011011001110110000101110011100000111001111100010100101000010011000000010100101100011001010111111000111010101111110010001010110110010010100000001010100000100000101011111000001000001111100000010100111110101010010110011000000010100100111010011011101001101100110001000111110000000000010011101100011111110101000100000101011000001110001111100001111011100110110000010010011000010101110011101100000110001010101001011111101000101101000001000111100011100011110000110000101111101010111010000001100000110110010101000011000110101110011011001101100100010101001110001000010110101100101010101001010000011001000000011100100110000010101011100010000000000000101011011100110100000011000100110101101110011010111000010001110011101101100000101111101010001110011100111111110011100011101011111100110101111100001001000101001010001000101010101101111011001110101100100000000001010001011000000110001000101111110100100000011110001110101100101101110001100001100001011010101000101100110110000000100011100000010111110101001000011110011111100000010110100110011101110100001011000001100101010110110010011110000100110100101000101110110101010101110011001100110111101001110111010101111000010111101011111100010110110100011010000110000001001101101100010010110111000010001101011011111101001000010010011010100010001100010100000000011000010100001110100011000100001010110100001100111101001001110111011010010111001001111001100100000010000101110101001101100101111011111011111101100110100001101011110100010001011000000111011000110001111001010000000101100111111111110111011110010010010110011111110000001011001010011011000001010011110111100111101001100101100000000000110001111111001010101011011011100011100110010010001010100100110001100010001100000111001011111010000001000011111000111011001011011000100110110001011001100110001110011011000001100011011010011111100011000110001110100111001101011110110111011000100000000110110011111001011001100010000000000110101110000010110001001001111011110000110000010000000101110001100001100010001101000100111001100011100000111010111001000110110011001101000110010000001000110000010101000100100110101011000011101101000111111000111000000100000111001010010101011111101111111001010100010000011010011111101110101111001111100000111001001'
# Perform the approximate entropy test
p_value_alice, result_alice = ApproximateEntropy.approximate_entropy_test(binary_data_alice, verbose=True, pattern_length=10)
p_value_bob, result_bob = ApproximateEntropy.approximate_entropy_test(binary_data_bob, verbose=True, pattern_length=10)


# Display the result
print("P-Value for Alice:", p_value_alice)
print("Result (True if the p-value >= 0.01, False otherwise):", result_alice)

print("P-Value for Bob:", p_value_bob)
print("Result (True if the p-value >= 0.01, False otherwise):", result_bob)

Approximate Entropy Test DEBUG BEGIN:
	Length of input:			 17576
	Length of each block:		 10
	ApEn(m):					 0.66335101501737
	xObs:						 1047.3948111526051
	P-Value:					 0.2988273469779222
DEBUG END.
Approximate Entropy Test DEBUG BEGIN:
	Length of input:			 17576
	Length of each block:		 10
	ApEn(m):					 0.6624728074170889
	xObs:						 1078.265564717687
	P-Value:					 0.11638988780253381
DEBUG END.
P-Value for Alice: 0.2988273469779222
Result (True if the p-value >= 0.01, False otherwise): True
P-Value for Bob: 0.11638988780253381
Result (True if the p-value >= 0.01, False otherwise): True
